### Прочитали fasta формат

In [1]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO
record = SeqIO.read("sequence.fasta", format="fasta")


In [37]:
record.seq[0:127]

Seq('MASFQSFGIPGQLEVIKKALDHVRVGVVITDPALEDNPIVYVNQGFVQMTGYET...DIT', SingleLetterAlphabet())

### Рассмотрим результат BLAST поиска

In [173]:
result_handle = open('W8ZR3WEN015-Alignment.xml')

In [174]:
from Bio.Blast import NCBIXML
blast_records = NCBIXML.parse(result_handle)

In [175]:
good_almns = {}
for blast_record in blast_records:
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            titles = alignment.title.split('>')
            for title in titles:
                t = title.split('|')
                ss =['517848989']
                for s in ss:
                    if t[1] == s:
                        print("****Alignment****")
                        print("sequence:", alignment.title)
               


Варианты, которые получлсь у меня и совпали со статьей 504822023, 505222758, 

In [1]:
good_almns

In [2]:
"""for blast_record in blast_records:
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            print("****Alignment****")
            print("sequence:", alignment.title)
            print(hsp.query_start, hsp.query_end, sep = ' ')
            print(hsp.query[0:75] + '...')"""

'for blast_record in blast_records:\n    for alignment in blast_record.alignments:\n        for hsp in alignment.hsps:\n            print("****Alignment****")\n            print("sequence:", alignment.title)\n            print(hsp.query_start, hsp.query_end, sep = \' \')\n            print(hsp.query[0:75] + \'...\')'

### Скачивание OneKP

In [4]:
from bs4 import BeautifulSoup   # Для обработки HTML
import requests # для получения кода страницы

Пропарсила страницу нашла нужные ссылки

In [5]:
data = requests.get('http://www.onekp.com/public_data.html')

In [9]:
soup = BeautifulSoup(data.text, 'html.parser')

In [1]:
print(soup.prettify())

Получим все ссылки

In [28]:
s= []
for string in soup.find_all('a'):
    s.append(string.get('href'))

Будем обращаться для скачивания по необходимым ссылкам

In [57]:
import urllib

In [55]:
res = [elem if elem.find("Trans-assembly")!=-1 else None for elem in s]

Попытка (удачная) скачать файл

In [59]:
url = 'http://206.12.96.204/onekp/URDJ-SOAPdenovo-Trans-assembly.fa.bz2'
urllib.request.urlretrieve(url, "URDJ.bz2")

('URDJ.bz2', <http.client.HTTPMessage at 0x7f476baf3e10>)

### Разделение файла с белками из Interpro

In [3]:
with open('proteins-matched_interpro.txt', 'r') as file:
    n = 0
    s_1, s_2, s_3, s_4 = [], [], [], []
    for line in file:
        if line.startswith('>'):
            n += 1
        if n <= 98328:
            s_1.append(line)
        elif 98328 < n <= 98328*2:
            s_2.append(line)
        elif 98328 * 2 < n <= 98328*3:
            s_3.append(line)
        else:
            s_4.append(line)
            
            

In [4]:
with open('protein_1.txt','w') as file:
    file.write(''.join(s_1))
with open('protein_2.txt','w') as file:
    file.write(''.join(s_2))
with open('protein_3.txt','w') as file:
    file.write(''.join(s_3))
with open('protein_4.txt','w') as file:
    file.write(''.join(s_4))

### Парсинг файлов mast и meme 

In [89]:
from bs4 import BeautifulSoup
with open("Without_Cys/mast_interpro_1_cl.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')
#print(soup.prettify())

In [90]:
motifs = soup.motifs.findAll('motif')

In [91]:
length = [motif['length'] for motif in motifs]

In [92]:
length

['35', '31', '29']

На 40 месте  в мотиве стоит **С** на который и нужно проверить или для Cysless на 24 месте не должен стоять.

In [93]:
sequences = soup.sequences.findAll('sequence')

In [94]:
import re
reg = re.compile('[^a-zA-Z ]')

Нашли все имена таких сиквенсов и также запсали данные о них в формати xml

In [95]:
Cys_seq_names = set()
Cys_seq = set()
for seq in sequences:
    for seg in seq.find_all('seg'):
        if bool(seg.find(idx = '0')) == True:
            if reg.sub('', seg.data.string)[int(seg.find(idx = '0')['pos']) - int(seg['start']) + 23] != 'C':
                Cys_seq_names.add(seq['name'])
                Cys_seq.add(seq)

In [59]:
with open('Without_Cys/Cysless_sequences_4.xml', 'w') as f:
    for i in Cys_seq:
        f.write(str(i) + '\n')

Все отсеившееся по нашему выбору сиквенсы запишем в отдельный файл

In [124]:
with open('protein_4.txt', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta.setdefault(name, []).append(line)

In [125]:
new_file = []
for name in Cys_seq_names:
    new_file.append('>' + name + '\n')
    for s in fasta.get('>' + name):
        new_file.append(s + '\n')

In [126]:
with open('Cysless_proteins_4.fasta', 'w') as f:
    f.write(''.join(new_file))

### Парсинг результатов pfam 

Сначала вручную удаляем информацию с *#* кроме последнего, с наименованием величин.  

In [1]:
import pandas as pd

In [115]:
columns = []
d = []
with open('Without_Cys/pfam_4_cl.txt', 'r') as f:
    for line in f:
        if line.startswith('<'):
            columns = line.split('> <')
        else:
            d.append(line.split())
columns[0] = columns[0].strip('<')
columns[-1] = columns[-1].strip('>\n')

In [116]:
data = pd.DataFrame(d, columns=columns)

In [117]:
data.head()

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan
0,M1FCN8,32,128,22,129,PF13426.7,PAS_9,Domain,11,103,104,76.6,1.5e-21,1,CL0183
1,M1FCN8,148,251,148,251,PF13426.7,PAS_9,Domain,1,104,104,64.4,9.1e-18,1,CL0183
2,M1FCN8,265,416,263,416,PF00990.21,GGDEF,Domain,3,161,161,125.5,1.5e-36,1,CL0276
3,V4Y023,135,236,134,236,PF13426.7,PAS_9,Domain,2,104,104,42.9,4.7e-11,1,CL0183
4,V4Y023,257,361,255,361,PF13426.7,PAS_9,Domain,3,104,104,76.0,2.3e-21,1,CL0183


### Идентификация LOV доменов

#### 1. Отбор по цистеину в первом мотиве

##### а) С цистеином

См. **Парсинг mast и meme**

##### б) Без цистеина

--//--

#### Отбор по P-value для каждого мотива и по нахождению в PAS fold

#### а) С Cys

In [175]:
from bs4 import BeautifulSoup
with open("With_Cys/Cys_sequences_4.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')

In [176]:
sequences = soup.sequences.findAll('sequence')

Проверим, что хотя бы какой-то сегмент имеет мотивы с нужным нам P-value и находятся в PAS fold

In [177]:
seqs = {}
for seq in sequences:
    for seg in seq.find_all('seg'):
        if bool(seg.find(idx = '0')) == True and bool(seg.find(idx = '1')) == True:
            hits = seg.find_all('hit')
            if float(hits[0]['pvalue']) <= 1e-15 and float(hits[1]['pvalue']) <= 1e-15:
                seqs.setdefault(seq['name'], []).append([int(hits[0]['pos']), int(hits[1]['pos'])])

In [82]:
seq_data = data.loc[data['seq id'] == 'A0A126WYA8']
df = seq_data.loc[seq_data['clan'] == 'CL0183']
df

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan
17843,A0A126WYA8,138,233,130,234,PF13426.7,PAS_9,Domain,8,103,104,59.4,3.3e-16,1,CL0183
17844,A0A126WYA8,419,511,410,514,PF13426.7,PAS_9,Domain,9,101,104,71.4,6e-20,1,CL0183


In [178]:
new_names = []
wrong = []
for k in seqs.keys():
    seq_data = data.loc[data['seq id'] == k]
    df = seq_data.loc[seq_data['clan'] == 'CL0183']
    b = False
    for hit in seqs[k]:
        for i in range(df.shape[0]):
            if int(df.iloc[i, 3]) - hit[0] <=5 and hit[1] + 43 - int(df.iloc[i, 4]) <= 1:
                b = True
    if not b:
        data.drop(data[data['seq id'] == k].index, inplace=True)
        wrong.append(k)
    else:
        new_names.append(k)    

In [181]:
with open('protein_4.txt', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta.setdefault(name, []).append(line)

In [182]:
new_file = []
for name in new_names:
    new_file.append('>' + name + '\n')
    for s in fasta.get('>' + name):
        new_file.append(s + '\n')

In [183]:
with open('With_Cys/final_proteins_4.fasta', 'w') as f:
    f.write(''.join(new_file))

#### б) без Cys

In [125]:
from bs4 import BeautifulSoup
with open("Without_Cys/Cysless_sequences_4.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')

In [126]:
sequences = soup.sequences.find_all('sequence')

In [127]:
seqs = {}
for seq in sequences:
    for seg in seq.find_all('seg'):
        if (bool(seg.find(idx = '0')) == True and bool(seg.find(idx = '1')) == True and bool(seg.find(idx = '2')) == True):
            hits = seg.find_all('hit')
            if (float(hits[0]['pvalue']) <= 1e-15 and float(hits[1]['pvalue']) <= 1e-15 and float(hits[2]['pvalue']) <= 1e-15):
                seqs.setdefault(seq['name'], []).append([int(hits[0]['pos']), int(hits[1]['pos']), 
                                                         int(hits[2]['pos'])])


In [128]:
new_names = []
wrong = []
for k in seqs.keys():
    seq_data = data.loc[data['seq id'] == k]
    df = seq_data.loc[seq_data['clan'] == 'CL0183']
    b = False
    for hit in seqs[k]:
        for i in range(df.shape[0]):
            if int(df.iloc[i, 3]) - hit[0] <= 2 and hit[2] + 28 - int(df.iloc[i, 4]) <= 10:
                b = True
    if not b:
        data.drop(data[data['seq id'] == k].index, inplace=True)
        wrong.append(k)
    else:
        new_names.append(k)    

In [129]:
len(Cys_seq_names)

839

In [130]:
len(seqs)

146

In [131]:
len(new_names)

146

In [132]:
with open('protein_4.txt', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta.setdefault(name, []).append(line)

In [133]:
new_file = []
for name in new_names:
    new_file.append('>' + name + '\n')
    for s in fasta.get('>' + name):
        new_file.append(s + '\n')

In [134]:
with open('Without_Cys/final_proteins_4_cl.fasta', 'w') as f:
    f.write(''.join(new_file))

In [41]:
with open('With_Cys/final_proteins.fasta', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta.setdefault(name, []).append(line)

In [42]:
with open('Without_Cys/final_proteins_cl.fasta', 'r') as f:
    fasta_cl = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line
        else:
            fasta_cl.setdefault(name, []).append(line)

In [43]:
repeat = list(fasta.keys() & fasta_cl.keys())
len(repeat)

0

Нашлись пересечения по наименованиям в final_proteins и final_proteins_cl удалим дубликаты, и проверим к какому именно классу они относятся

In [35]:
for r in repeat:
    fasta_cl.pop(r)

In [39]:
new_file = []
for name in fasta_cl.keys():
    new_file.append(name + '\n')
    for s in fasta_cl.get(name):
        new_file.append(s + '\n')

In [40]:
with open('Without_Cys/final_proteins_cl.fasta', 'w') as f:
    f.write(''.join(new_file))

 В итоге получили файл final без повторов, и уменьшили количество сиквенсов в final_cl.

Теперь рассмотрим эффекторы, используя данные pfam полученные ранее

### Эффекторы LOV

#### C Cys

In [80]:
columns = []
d = []
with open('With_Cys/pfam_3.txt', 'r') as f:
    for line in f:
        if line.startswith('<'):
            columns = line.split('> <')
        else:
            d.append(line.split())
columns[0] = columns[0].strip('<')
columns[-1] = columns[-1].strip('>\n')

In [81]:
data = pd.DataFrame(d, columns=columns)

Загрузка df из csv файла

In [3]:
data = pd.read_csv('With_Cys/domains_1.csv')

In [4]:
data.drop('Unnamed: 0', axis = 1, inplace = True)
data.head()

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan,kingdom
0,A0A0J6VGA8,62,156,52,157,PF13426.7,PAS_9,Domain,9,103,104,69.7,2.100000e-19,1,CL0183,Bacteria Proteobacteria
1,A0A0J6VGA8,175,235,173,237,PF00512.25,HisKA,Domain,3,64,67,34.0,2.200000e-08,1,CL0025,Bacteria Proteobacteria
2,A0A0J6VGA8,280,402,279,405,PF02518.26,HATPase_c,Domain,2,109,112,49.2,6.500000e-13,1,CL0025,Bacteria Proteobacteria
3,A0A0J6VGA8,427,537,427,538,PF00072.24,Response_reg,Domain,1,111,112,67.4,1.100000e-18,1,CL0304,Bacteria Proteobacteria
4,A0A059UGE7,331,427,324,428,PF13426.7,PAS_9,Domain,7,103,104,69.9,1.900000e-19,1,CL0183,Eukaryota Viridiplantae


In [82]:
from bs4 import BeautifulSoup
with open("With_Cys/Cys_sequences_3.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')

In [83]:
sequences = soup.sequences.findAll('sequence')

Для каждого наименования в final-ах, будем рассматривать расположение LOV (по наличию двух мотивов внутри) и ближайшие к нему эффекторы

In [84]:
'''Доступ к Uniprot'''
from Bio import ExPASy
from Bio import SwissProt

In [42]:
handle = ExPASy.get_sprot_raw('A0A126WYA8')
record = SwissProt.read(handle)

In [43]:
record.organism_classification[0] + ' ' + record.organism_classification[1]

'Eukaryota Viridiplantae'

Разделим все наименования на (растения), (грибы), (бактерии), (археи), (протисты) 

In [85]:
data['kingdom'] = None

In [8]:
def kingdom(s):
    handle = ExPASy.get_sprot_raw(s[0])
    try:
        record = SwissProt.read(handle)
        return record.organism_classification[0] + ' ' + record.organism_classification[1]
    except ValueError:
        return None

In [ ]:
from tqdm import trange

In [114]:
for i in trange(1034, 5065):
    data.iloc[i, 15] = kingdom(data.iloc[i,:])







  0%|          | 0/4031 [00:00<?, ?it/s]





  0%|          | 1/4031 [00:00<33:39,  2.00it/s]





  0%|          | 2/4031 [00:01<39:54,  1.68it/s]





  0%|          | 3/4031 [00:01<38:54,  1.73it/s]





  0%|          | 4/4031 [00:02<43:32,  1.54it/s]





  0%|          | 5/4031 [00:03<40:42,  1.65it/s]





  0%|          | 6/4031 [00:03<44:59,  1.49it/s]





  0%|          | 7/4031 [00:04<38:59,  1.72it/s]





  0%|          | 8/4031 [00:04<34:38,  1.94it/s]





  0%|          | 9/4031 [00:05<31:34,  2.12it/s]





  0%|          | 10/4031 [00:05<29:29,  2.27it/s]





  0%|          | 11/4031 [00:05<27:38,  2.42it/s]





  0%|          | 12/4031 [00:06<35:43,  1.88it/s]





  0%|          | 13/4031 [00:07<32:24,  2.07it/s]





  0%|          | 14/4031 [00:07<29:40,  2.26it/s]





  0%|          | 15/4031 [00:07<27:45,  2.41it/s]





  0%|          | 16/4031 [00:08<32:37,  2.05it/s]





  0%|          | 17/4031 [00:08<35:36,  1.88it/s]





  0%|          | 18/40

  3%|▎         | 111/4031 [01:20<44:00,  1.48it/s]





  3%|▎         | 112/4031 [01:21<37:50,  1.73it/s]





  3%|▎         | 113/4031 [01:21<36:26,  1.79it/s]





  3%|▎         | 114/4031 [01:22<38:25,  1.70it/s]





  3%|▎         | 115/4031 [01:22<34:11,  1.91it/s]





  3%|▎         | 116/4031 [01:23<33:48,  1.93it/s]





  3%|▎         | 117/4031 [01:23<37:00,  1.76it/s]





  3%|▎         | 118/4031 [01:24<38:26,  1.70it/s]





  3%|▎         | 119/4031 [01:25<55:05,  1.18it/s]





  3%|▎         | 120/4031 [01:26<45:29,  1.43it/s]





  3%|▎         | 121/4031 [01:26<44:41,  1.46it/s]





  3%|▎         | 122/4031 [01:27<42:10,  1.54it/s]





  3%|▎         | 123/4031 [01:28<45:10,  1.44it/s]





  3%|▎         | 124/4031 [01:28<38:42,  1.68it/s]





  3%|▎         | 125/4031 [01:29<34:12,  1.90it/s]





  3%|▎         | 126/4031 [01:29<39:02,  1.67it/s]





  3%|▎         | 127/4031 [01:30<43:02,  1.51it/s]





  3%|▎         | 128/4031 [01:31<43:29,  1.50it/

RemoteDisconnected: Remote end closed connection without response

In [113]:
data.iloc[1034,15]

In [79]:
tfile = open('With_Cys/domains_2.csv', 'a')
tfile.write(data.to_csv())
tfile.close()

In [87]:
data

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan,kingdom
0,A0A2R6DGK3,82,172,73,183,PF00072.24,Response_reg,Domain,11,103,112,55.0,7.9e-15,1,CL0304,None
1,A0A2R6DGK3,216,320,214,320,PF13426.7,PAS_9,Domain,3,104,104,74.6,6.1e-21,1,CL0183,None
2,A0A2R6DGK3,364,500,363,501,PF13185.6,GAF_2,Domain,2,137,138,46.5,4.2e-12,1,CL0161,None
3,A0A2R6DGK3,512,667,512,668,PF15915.5,BAT,Domain,1,155,156,121.1,3.8e-35,1,No_clan,None
4,A0A2R6DGK3,674,726,674,726,PF04967.12,HTH_10,Domain,1,53,53,67.0,9.2e-19,1,CL0123,None
5,A0A2S5EYS5,16,121,15,122,PF13426.7,PAS_9,Domain,2,103,104,78.5,3.7e-22,1,CL0183,None
6,A0A2R6KDK3,149,253,147,253,PF13426.7,PAS_9,Domain,3,104,104,69.2,3e-19,1,CL0183,None
7,A0A2R6KDK3,268,326,267,328,PF00512.25,HisKA,Domain,2,65,67,41.9,7.5e-11,1,CL0025,None
8,A0A2R6KDK3,366,468,366,471,PF02518.26,HATPase_c,Domain,1,109,112,52.8,4.9e-14,1,CL0025,None
9,A0A2N8SZ02,16,121,15,121,PF13426.7,PAS_9,Domain,2,104,104,73.3,1.5e-20,1,CL0183,None


In [86]:
data.shape

(5065, 16)

In [ ]:
data.groupby(by = 'kingdom').groups